In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale

In [2]:
from samplesizelib.linear.statistical import LagrangeEstimator
from samplesizelib.linear.statistical import LikelihoodRatioEstimator
from samplesizelib.linear.statistical import WaldEstimator

from samplesizelib.linear.heuristic import CrossValidationEstimator
from samplesizelib.linear.heuristic import BootstrapEstimator
from samplesizelib.linear.heuristic import LogisticRegressionEstimator

from samplesizelib.linear.bayesian import APVCEstimator
from samplesizelib.linear.bayesian import ACCEstimator
from samplesizelib.linear.bayesian import ALCEstimator
from samplesizelib.linear.bayesian import MaxUtilityEstimator
from samplesizelib.linear.bayesian import KLEstimator

from samplesizelib.linear.models import RegressionModel
from samplesizelib.linear.models import LogisticModel

In [3]:
NAME_TO_MODEL = {
    'LagrangeEstimator': LagrangeEstimator, 
    'LikelihoodRatioEstimator': LikelihoodRatioEstimator, 
    'WaldEstimator': WaldEstimator, 
    'CrossValidationEstimator': CrossValidationEstimator, 
    'BootstrapEstimator': BootstrapEstimator, 
    'LogisticRegressionEstimator': LogisticRegressionEstimator, 
    'APVCEstimator': APVCEstimator, 
    'ACCEstimator': ACCEstimator, 
    'ALCEstimator': ALCEstimator, 
    'ALCEstimator': ALCEstimator, 
    'MaxUtilityEstimator': MaxUtilityEstimator, 
    'KLEstimator': KLEstimator
}

config = {
    'LagrangeEstimator': {'epsilon': 0.3, 
                          'alpha': 0.05, 
                          'beta': 0.05},
    'LikelihoodRatioEstimator': {'epsilon': 0.3, 
                                 'alpha': 0.05, 
                                 'beta': 0.05},
    'WaldEstimator': {'epsilon': 0.3, 
                      'alpha': 0.05, 
                      'beta': 0.05},
    'BootstrapEstimator': {'averaging': 10,
                           'epsilon': 0.5},
    'CrossValidationEstimator': {'averaging': 10,
                                 'test_size': 0.5, 
                                 'epsilon': 0.05},
    'APVCEstimator': {'averaging': 10,
                      'epsilon': 0.5},
    'ACCEstimator': {'averaging': 10,
                     'length': 0.25,
                     'alpha': 0.05},
    'ALCEstimator': {'averaging': 10,
                     'length': 0.5,
                     'alpha': 0.05},
    'MaxUtilityEstimator': {'averaging': 10,
                            'c': 0.5},
    'KLEstimator': {'averaging': 5,
                    'epsilon': 0.5}
}

# Abalone

Load dataset

In [4]:
dataset = pd.read_csv('datasets/abalone.csv')

y = dataset.iloc[:, dataset.columns == 'answer'].values
X = dataset.iloc[:, dataset.columns != 'answer'].values

X = scale(X)
X = np.hstack((X, np.ones([len(X), 1])))

In [6]:
statmodel = LogisticRegressionEstimator

models = dict()
for key in config:
    models[key] = NAME_TO_MODEL[key](statmodel, **config[key])

In [7]:
result = dict()

for i, key in enumerate(models):
    result[key] = models[key](X, y)

NameError: name 'self' is not defined